In [ ]:

from dotenv import load_dotenv

load_dotenv()

import os
print(os.environ)
print(os.getenv("GOOGLE_API_KEY"))
print(os.getenv("LANGSMITH_API_KEY"))
print(os.getenv("SERPAPI_API_KEY"))

from langchain_teddynote import logging

logging.langsmith("rag1")


In [ ]:
from pydantic import BaseModel, Field


# 이메일 본문으로부터 주요 엔티티 추출
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    company: str = Field(description="메일을 보낸 사람의 회사 정보")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [ ]:
import pickle as pkl

with open("email.pkl", "rb") as buf:
    data=pkl.load(buf)

email_conversation = data["email"]
prompt = data["prompt"]
answer = data["answer"]

https://serpapi.com/integrations/python

In [ ]:
#!pip install serpapi
# ImportError: Could not import serpapi python package. Please install it with `pip install google-search-results`.
#!pip install google-search-results


In [ ]:
from langchain_community.utilities import SerpAPIWrapper

params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}

search = SerpAPIWrapper(params=params)

In [ ]:
answer.email

In [ ]:
query = f"{answer.person} {answer.company} {answer.email}"
query

In [ ]:
search_result = search.run(query)

In [ ]:
print(type(search_result))
print(len(search_result))
print(search_result)

In [ ]:
search_result = eval(search_result)


In [ ]:
print(type(search_result))
print(len(search_result))
print(search_result)

In [ ]:
for i, aresult in enumerate(search_result):
    print(i, aresult)

In [ ]:
# 검색 결과
search_result_string = "\n".join(search_result)

In [ ]:
print(type(search_result_string))
print(len(search_result_string))
print(search_result_string)

In [ ]:
answer

In [ ]:
from langchain_core.prompts import PromptTemplate

report_prompt = PromptTemplate.from_template(
    """당신은 이메일의 주요 정보를 바탕으로 요약 정리해 주는 전문가 입니다.
당신의 임무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
주어진 정보를 기반으로 양식(format)에 맞추어 요약을 작성해 주세요.

#Information:
- Sender: {sender}
- Additional Information about sender: {additional_information}
- Company: {company}
- Email: {email}
- Subject: {subject}
- Summary: {summary}
- Date: {date}

#Format(in markdown format):
🙇‍♂️ 보낸 사람:
- (보낸 사람의 이름, 회사 정보)

📧 이메일 주소:
- (보낸 사람의 이메일 주소)

😍 보낸 사람과 관련하여 검색된 추가 정보:
- (검색된 추가 정보)

✅ 주요 내용:
- (이메일 제목, 요약)

⏰ 일정:
- (미팅 날짜 및 시간)

#Answer:"""
)

In [ ]:

#from langchain_openai import ChatOpenAI

#llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.,
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
    # other params...
)


In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
## LCEL 구조

# chain = prompt | llm | output_parser


report_chain = (
    report_prompt | llm | StrOutputParser()
)

In [ ]:
report_response = report_chain.invoke(
    {
        "sender": answer.person,
        "additional_information": search_result_string,
        "company": answer.company,
        "email": answer.email,
        "subject": answer.subject,
        "summary": answer.summary,
        "date": answer.date,
    }
)

In [ ]:
print(report_response)

In [ ]:
import pickle as pkl

data={ 
    "email":email_conversation,
    "prompt":prompt,
    "answer":answer,
    "search_result_string":search_result_string,
    "report_prompt":report_prompt,
    "report_response":report_response
        }

with open("email_report.pkl", "wb", ) as buf:
    pkl.dump(data, buf)
